In [46]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))
# print(tf.test.is_built_with_cuda())
import numpy as np
from tensorflow import keras
import random
import string
import pandas as pd
import re
from tqdm import tqdm
import pickle

# import wget


In [62]:
# !curl -O http://www.manythings.org/anki/fra-eng.zip
# with zipfile.ZipFile("fra-eng.zip","r") as zip_ref:
#     zip_ref.extractall("targetdir")

txt = open("D:/Mithma/SLIIT/Y4/CDAP/word_frequency_list_2M.si", "r", encoding="utf8")
dataset= txt.read()


# url='https://drive.google.com/file/d/1SeqwPPUFKL8t72j5VkhNWTvY84cnaAgX/view?usp=sharing'
# url='https://drive.google.com/uc?id=' + url.split('/')[-2]
# dataset = pd.read_csv(url)
# print(dataset)




In [64]:
#Dataset preview
print(dataset)


             මේ DET 1067105
0              ඒ ABB 946049
1            ඇති NIP 567890
2              සහ CC 518043
3              හා CC 511663
4             එක NUM 491217
...                     ...
2138015       අනිනවාය VNF 1
2138016    නොදැමුවෙමි VFM 1
2138017    කියවහල්ලකො NNP 1
2138018  ගෑනුලමයෙකුගේ NNC 1
2138019       පල්ක්කු NNC 1

[2138020 rows x 1 columns]


In [66]:
len(dataset) #length of the dataset

2138020

In [67]:
dataset.info() #dataset information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138020 entries, 0 to 2138019
Data columns (total 1 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   මේ DET 1067105  object
dtypes: object(1)
memory usage: 16.3+ MB


In [48]:
num_samples = 1000000  # Number of samples to train on.
data_path = "D:/Mithma/SLIIT/Y4/CDAP/word_frequency_list_2M.si"  # Path to the data txt file on disk.
print(data_path)

D:/Mithma/SLIIT/Y4/CDAP/word_frequency_list_2M.si


In [49]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.
output_dim = 64


In [50]:
# unique_list = []

# c1 = []
# with open('D:/Mithma/SLIIT/Y4/CDAP/word_frequency_list_2M.si', 'r', encoding="utf-8") as wordList:
#     question_list = []
#     answer_list = []
#     for line in wordList.readlines():
#         question = line.split(' ')
#         question_list.append(question)
        

In [51]:
# Vectorize the data.
input_texts = []
target_texts = []
input_ =[]
tar_ =[]
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

token_list=[ 
'අ', 'ආ', 'ඇ', 'ඈ', 'ඉ', 'ඊ', 'උ', 'ඌ', 'එ', 'ඒ', 'ඓ', 'ඔ', 'ඕ', 'ඖ', 'ඍ',
'ක', 'කා', 'කැ', 'කෑ', 'කි', 'කී', 'කු', 'කූ', 'කෙ', 'කේ', 'කෛ', 'කො', 'කෝ', 'කෞ', 
'ඛ', 'ඛා', 'ඛැ' , 'ඛෑ', 'ඛි' , 'ඛී' , 'ඛු' , 'ඛූ', 'ඛෙ', 'ඛේ', 'ඛෛ', 'ඛො', 'ඛෝ', 'ඛෞ',
'ග', 'ගා', 'ගැ', 'ගෑ','ගි', 'ගී', 'ගු' , 'ගූ', 'ගෙ', 'ගේ', 'ගෛ', 'ගො' ,'ගෝ' ,'ගෞ',
'ඝ', 'ඝා', 'ඝැ', 'ඝෑ', 'ඝි', 'ඝී', 'ඝු', 'ඝූ', 'ඝෙ', 'ඝේ', 'ඝෛ', 'ඝො', 'ඝෝ', 'ඝෞ',
'ඞ', 'ඞා', 'ඞැ', 'ඞෑ','ඞි','ඞී', 'ඞු','ඞූ', 'ඞෙ', 'ඞේ', 'ඞෛ', 'ඞො','ඞෝ','ඞෞ',
'ඟ', 'ඟා', 'ඟැ', 'ඟෑ', 'ඟි', 'ඟී', 'ඟු', 'ඟූ', 'ඟෙ','ඟේ','ඟෛ','ඟො','ඟෝ','ඟෞ',
'ච', 'චා', 'චැ', 'චෑ', 'චි', 'චී', 'චු', 'චූ', 'චෙ','චේ','චෛ','චො','චෝ','චෞ',
'ඡ', 'ඡා', 'ඡැ', 'ඡෑ', 'ඡි', 'ඡී', 'ඡු', 'ඡූ', 'ඡෙ','ඡේ','ඡෛ','ඡො','ඡෝ','ඡෞ',
'ජ', 'ජා', 'ජැ', 'ජෑ', 'ජි', 'ජී', 'ජු', 'ජූ', 'ජෙ','ජේ','ජෛ','ජො','ජෝ','ජෞ',
'ඣ', 'ඣා', 'ඣැ', 'ඣෑ', 'ඣි', 'ඣී', 'ඣු', 'ඣූ', 'ඣෙ','ඣේ','ඣෛ','ඣො','ඣෝ','ඣෞ',
'ඤ', 'ඤා', 'ඤැ', 'ඤෑ', 'ඤි', 'ඤී', 'ඤු', 'ඤූ', 'ඤෙ','ඤේ','ඤෛ','ඤො','ඤෝ','ඤෞ',
'ඥ', 'ඥා', 'ඥැ', 'ඥෑ', 'ඥි', 'ඥී', 'ඥු', 'ඥූ', 'ඥෙ','ඥේ','ඥෛ','ඥො','ඥෝ','ඥෞ',
    'ඦ', 'ඦා', 'ඦැ', 'ඦෑ', 'ඦි', 'ඦී', 'ඦු', 'ඦූූ', 'ඦෙ','ඦේ','ඦෛ','ඦො','ඦෝ','ඦෞ',
    'ට', 'ටා', 'ටැ', 'ටෑ', 'ටි', 'ටී', 'ටු', 'ටූ', 'ටෙ','ටේ','ටෛ','ටො','ටෝ','ටෞ',
    'ඨ', 'ඨා', 'ඨැ', 'ඨෑ', 'ඨි', 'ඨී', 'ඨු', 'ඨූ', 'ඨෙ','ඨේ','ඨෛ','ඨො','ඨෝ','ඨෞ',
    'ඩ', 'ඩා', 'ඩැ', 'ඩෑ', 'ඩි', 'ඩී', 'ඩු', 'ඩූ', 'ඩෙ','ඩේ','ඩෛ','ඩො','ඩෝ','ඩෞ',
    'ඪ', 'ඪා', 'ඪැ', 'ඪෑ', 'ඪි', 'ඪී', 'ඪු', 'ඪූ', 'ඪෙ','ඪේ','ඪෛ','ඪො','ඪෝ','ඪෞ',
  'ණ', 'ණා', 'ණැ', 'ණෑ', 'ණි', 'ණී', 'ණු', 'ණූ', 'ණෙ','ණේ','ණෛ','ණො','ණෝ','ණෞ',
    'ඬ', 'ඬා', 'ඬැ', 'ඬෑ', 'ඬි', 'ඬී', 'ඬු', 'ඬූ', 'ඬෙ','ඬේ','ඬෛ','ඬො','ඬෝ','ඬෞ',
      'ත', 'තා', 'තැ', 'තෑ', 'ති', 'තී', 'තු', 'තූ', 'තෙ','තේ','තෛ','තො','තෝ','තෞ',
        'ථ​', 'ථා', 'ථැ', 'ථෑ', 'ථි', 'ථී', 'ථු', 'ථූ', 'ථෙ','ථේ','ථෛ','ථො','ථෝ','ථෞ',
          'ද​​', 'දා​', 'දැ', 'දෑ', 'දි', 'දී', 'දු', 'දූ', 'දෙ','දේ','දෛ','දො','දෝ','දෞ',
            'ධ​​', 'ධා​', 'ධැ', 'ධෑ', 'ධි', 'ධී', 'ධු', 'ධූ', 'ධෙ','ධේ','ධෛ','ධො','ධෝ','ධෞ',
              'න​​​', 'නා', 'නැ', 'නෑ', 'නි', 'නී', 'නු', 'නූ', 'නෙ','නේ','නෛ','නො','නෝ','නෞ',
                'ඳ​​​', 'ඳා', 'ඳැ', 'ඳෑ', 'ඳි', 'ඳී', 'ඳු', 'ඳූ', 'ඳෙ','ඳේ','ඳෛ','ඳො','ඳෝ','ඳෞ',
                  'ප​​​​', 'පා', 'පැ', 'පෑ', 'පි', 'පී', 'පු', 'පූ', 'පෙ','පේ','පෛ','පො','පෝ','පෞ',
 'ඵ​​​​', 'ඵා', 'ඵැ', 'ඵෑ', 'ඵි', 'ඵී', 'ඵු', 'ඵූ', 'ඵෙ','ඵේ','ඵෛ','ඵො','ඵෝ','ඵෞ',
   'බ​​​​', 'බා', 'බැ', 'බෑ', 'බි', 'බී', 'බු', 'බූ', 'බෙ','බේ','බෛ','බො','බෝ','බෞ',
     'භ​​​​', 'භා', 'භැ', 'භෑ', 'භි', 'භී', 'භු', 'භූ', 'භෙ','භේ','භෛ','භො','භෝ','භෞ',
  'ම​​​​', 'මා', 'මැ', 'මෑ', 'මි', 'මී', 'මු', 'මූ', 'මෙ','මේ','මෛ','මො','මෝ','මෞ', 
 'ඹ​​​​', 'ඹා', 'ඹැ', 'ඹෑ', 'ඹි', 'ඹී', 'ඹු', 'ඹූ', 'ඹෙ','ඹේ','ඹෛ','ඹො','ඹෝ','ඹෞ',
  'ය​​​​', 'යා', 'යැ', 'යෑ', 'යි', 'යී', 'යු', 'යූ', 'යෙ','යේ','යෛ','යො','යෝ','යෞ',
  'ර​​​​', 'රා', 'රැ', 'රෑ', 'රි', 'රී', 'රු', 'රූ', 'රෙ','රේ','රෛ','රො','රෝ','රෞ',
  'ල​​​​', 'ලා', 'ලැ', 'ලෑ', 'ලි', 'ලී', 'ලු', 'ලූ', 'ලෙ','ලේ','ලෛ','ලො','ලෝ','ලෞ',
  'ළ​​​​', 'ළා', 'ළැ', 'ළෑ', 'ළි', 'ළී', 'ළු', 'ළූ', 'ළෙ','ළේ','ළෛ','ළො','ළෝ','ළෞ',
  'ව​​​​', 'වා', 'වැ', 'වෑ', 'වි', 'වී', 'වු', 'වූ', 'වෙ','වේ','වෛ','වො','වෝ','වෞ',
    'හ​​​​', 'හා', 'හැ', 'හෑ', 'හි', 'හී', 'හු', 'හූ', 'හෙ','හේ','හෛ','හො','හෝ','හෞ',
 'ශ​​​​', 'ශා', 'ශැ', 'ශෑ', 'ශි', 'ශී', 'ශු', 'ශූ', 'ශෙ','ශේ','ශෛ','ශො','ශෝ','ශෞ',
   'ෂ​​​​', 'ෂා', 'ෂැ', 'ෂෑ', 'ෂි', 'ෂී', 'ෂු', 'ෂූ', 'ෂෙ','ෂේ','ෂෛ','ෂො','ෂෝ','ෂෞ',
     'ස​​​​', 'සා', 'සැ', 'සෑ', 'සි', 'සී', 'සු', 'සූ', 'සෙ','සේ','සෛ','සො','සෝ','සෞ',
  'ෆ​​​​', 'ෆා', 'ෆැ', 'ෆෑ', 'ෆී', 'ෆි', 'ෆු', 'ෆූ', 'ෆෙ','ෆේ','ෆෛ','ෆො','ෆෝ','ෆෞ',
    'ක්‍ෂ​​​​', 'ක්‍ෂා', 'ක්‍ෂැ', 'ක්‍ෂෑ', 'ක්‍ෂි', 'ක්‍ෂී', 'ක්‍ෂු', 'ක්‍ෂූ', 'ක්‍ෂෙ','ක්‍ෂේ','ක්‍ෂෛ','ක්‍ෂො','ක්‍ෂෝ','ක්‍ෂෞ',
      'ා​​​​', 'ැ', 'ෑ','ි','ී','ු','ූ','ෙ',  'ා​​​​', 'ැ', 'ෑ','ි','ී','ු','ූ','ෙ','ේ','ෛ','ො','ෝ','ෞ', 'ෘ','ෲ','ෟ','ෳ'


for line in lines[: min(num_samples, len(lines) - 1)]:
    for _ in range(5):
        
        input_text, target_text, _ = line.split(" ")

        # input_text = re.sub(r'[^a-zA-Z ]+', '', input_text)
        target_text = "\t" + input_text + "\n"
        for i in range(np.random.choice(np.arange(0, 2), p=[0.1, 0.9])):
            input_text = input_text.replace(random.choice(list(input_text)),random.choice(token_list))

        input_texts.append(input_text.lower())
        target_texts.append(target_text)
        for char in input_text.lower():
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
              

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    # encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    # decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    # decoder_target_data[i, t:, target_token_index[" "]] = 1.0




Number of samples: 6000
Number of unique input tokens: 69
Number of unique output tokens: 60
Max sequence length for inputs: 15
Max sequence length for outputs: 16


In [52]:
input_token_index

{'ං': 0,
 'අ': 1,
 'ආ': 2,
 'ඇ': 3,
 'ඈ': 4,
 'ඉ': 5,
 'ඊ': 6,
 'උ': 7,
 'ඌ': 8,
 'ඍ': 9,
 'එ': 10,
 'ඒ': 11,
 'ඓ': 12,
 'ඔ': 13,
 'ඕ': 14,
 'ඖ': 15,
 'ක': 16,
 'ඛ': 17,
 'ග': 18,
 'ඝ': 19,
 'ඞ': 20,
 'ඟ': 21,
 'ච': 22,
 'ඡ': 23,
 'ජ': 24,
 'ඣ': 25,
 'ඤ': 26,
 'ඥ': 27,
 'ඦ': 28,
 'ට': 29,
 'ඨ': 30,
 'ඩ': 31,
 'ණ': 32,
 'ත': 33,
 'ථ': 34,
 'ද': 35,
 'ධ': 36,
 'න': 37,
 'ඳ': 38,
 'ප': 39,
 'බ': 40,
 'භ': 41,
 'ම': 42,
 'ඹ': 43,
 'ය': 44,
 'ර': 45,
 'ල': 46,
 'ව': 47,
 'ශ': 48,
 'ෂ': 49,
 'ස': 50,
 'හ': 51,
 'ළ': 52,
 '්': 53,
 'ා': 54,
 'ැ': 55,
 'ෑ': 56,
 'ි': 57,
 'ී': 58,
 'ු': 59,
 'ූ': 60,
 'ෘ': 61,
 'ෙ': 62,
 'ේ': 63,
 'ෛ': 64,
 'ො': 65,
 'ෝ': 66,
 'ෞ': 67,
 '\u200d': 68}

In [53]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True, dropout=0.4)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [54]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [55]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=100,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.LearningRateScheduler(
            lambda epoch: 1e-3 * 10 ** (epoch / 30)
        )
    ]
)
# Save model
model.save("auto.h5")

Epoch 1/5
75/75 [==============================] - 8s 57ms/step - loss: 1.2150 - accuracy: 0.0657 - val_loss: 1.2514 - val_accuracy: 0.0827
Epoch 2/5
75/75 [==============================] - 3s 45ms/step - loss: 1.1151 - accuracy: 0.0780 - val_loss: 1.2242 - val_accuracy: 0.0838
Epoch 3/5
75/75 [==============================] - 3s 41ms/step - loss: 1.1002 - accuracy: 0.0813 - val_loss: 1.2235 - val_accuracy: 0.0841
Epoch 4/5
75/75 [==============================] - 3s 40ms/step - loss: 1.0858 - accuracy: 0.0843 - val_loss: 1.2261 - val_accuracy: 0.0807
Epoch 5/5
75/75 [==============================] - 3s 41ms/step - loss: 1.0759 - accuracy: 0.0905 - val_loss: 1.1812 - val_accuracy: 0.0990


In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 69)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 60)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        101376      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

In [57]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [58]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
#model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_5")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [59]:
test_text = "ළමයා"

encoder_test_data = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")

for t, char in enumerate(test_text):
    encoder_test_data[0, t, input_token_index[char]] = 1.0

decoded_sentence = decode_sequence(encoder_test_data)
print(decoded_sentence)

1/1 [==============================] - 0s 24ms/step
කම



In [60]:
out = pd.DataFrame(columns = ["input", "decoded", "target"])
tt = 0
for seq_index in tqdm(range(57300, 57400)):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    out.loc[len(out.index)] = [input_texts[seq_index], decoded_sentence, target_texts[seq_index]]

  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
out.tail(20)
np.shape(input_seq)
tt

In [ ]:
f = open('var.pkl','wb')
jj = {"input_token_index":input_token_index, "target_token_index":target_token_index, "num_decoder_tokens": num_decoder_tokens, "max_encoder_seq_length":max_encoder_seq_length, "num_encoder_tokens": num_encoder_tokens}
pickle.dump(jj, f)
f.close()